In [30]:
import pandas as pd
from datetime import timedelta
import os

# 1. Load data

In [2]:
agg_profile_views = pd.read_csv('/Users/jimchu/Desktop/DOWN_App/agg_profile_views.csv')
dim_bad_actors = pd.read_csv('/Users/jimchu/Desktop/DOWN_App/dim_bad_actors.csv')
dim_users = pd.read_csv('/Users/jimchu/Desktop/DOWN_App/dim_users.csv')

# 2. Processing

## 2.1 `dim_users`
> - Had duplicate `user_id` with different `created_at`
>   - keep `min(created_at)` as `created_at`.

> - Had duplicate `user_id` with different `gender`
>   - drop users who has different genders as couldn't define through the given information.

> - Only had 1 timezone (UTC)

In [4]:
## keep min(created_at)
dim_users_clean = dim_users.groupby(['user_id', 'gender'], as_index=False).created_at.min()

## `create_at` processing
dim_users_clean[['created_at_dt', 'created_at_timezone']] = dim_users_clean['created_at'].str.rsplit(' ', n = 1, expand = True)
dim_users_clean['created_at_dt'] = dim_users_clean['created_at_dt'].apply(lambda x: x.rsplit(' ')[0])

## user_id lists of whom has different genders
MultiGender_UsersList = dim_users_clean.groupby(['user_id'], as_index=False).gender.nunique()
MultiGender_UsersList = MultiGender_UsersList[MultiGender_UsersList.gender > 1].user_id.unique()

### generate clean table ###
dim_users_clean = dim_users_clean[dim_users_clean.user_id.isin(MultiGender_UsersList) == False]
dim_users_clean = dim_users_clean[['user_id', 'gender', 'created_at_dt']]

In [5]:
dim_users_clean

,user_id,gender,created_at_dt
0,+++uJ1aPqZNAZYyXGqRh3xUhLfWhIC+Bcpe2powL+Ho=,male,2024-08-06
1,++/HNxcsuGmDh9wapV3frR+m5dyPGVaDUGDSbtFR/WM=,male,2024-08-26
2,++2WCrmXuIC36VhFgInvwWITW7GQEd9MM9o1m86eyrY=,male,2023-06-08
3,++367sM0W1OoxV0UyLBGNznuv7fLpkRV7fm2MQDCLzw=,male,2024-08-17
4,++4sGpSOQ1CUoNXUieM4Oh4bCzPMMz4bmwnBgZo2WOM=,male,2024-08-10
...,...,...,...
183978,zzseEVXFw16OsiRWWeqwoqr1WB4LHOua2XIZvJoEKSE=,male,2023-08-26
183979,zzukoZ9xdszYnhPnVA/8RcJswja5pnuNiVa4iQ8Kp2w=,male,2024-03-14
183980,zzvCOH5MnI/QpulSZsnzjFUi2336aiNs2Q1Y8u9+44s=,male,2024-08-03
183981,zzx0i1OlOREB+L/NuZwbtCcgRuGZUfB/PunZMITloGg=,male,2024-05-15


## 2.2 `dim_bad_actors`
- No duplicated data

In [6]:
dim_bad_actors.user_id.value_counts()

upUExV12F9pCHGL0imzgPme0D182a4equzoTr8FzjiU=    1
wbwfWDJ1VdnNKi0rpUXtXa7WRE7oRlzV5ma+ZfLa+Bo=    1
jR+j7kQwKE3f1Lz6/Uz7165wS6KqlYlfI284HPbJEpM=    1
2yjCoOy0jOFtQLBl5pLcm5YFW1NykUQoozN/ogMMT1I=    1
BaF5vsz1irQqwwYwKSgNYdJMK/3HQGDhx5anUhGhHgE=    1
                                               ..
j7v87Ey14pIEnG3puYr4moQUbP6h11B4wfEe3RTpeaY=    1
BNMz6Jpkwno/8EStaKS89KzjaST60Pm8YLZSZdoyqGE=    1
4WpcpxIgX0npHsfZG8zLI3nG7bwFRJCLjUouk9UXFEo=    1
gbkFaH8fjf61yJVRmfwrNXi5fQhok6Vggu0qsUUGPZw=    1
Xohvx8LESgjL3mJHYtPPvmlFnGCvBGV8fWhkUtDrY4U=    1
Name: user_id, Length: 9137, dtype: int64

In [7]:
## add "Bad Actors marker"
dim_bad_actors['IfBadActor'] = 1

## 2.3 `agg_profile_views`
- No duplicated data
- No `user_id` == `viewed_user_id`
- `filter_type` has Null data
    - should not have the null value, so exclude the row with Null

In [13]:
agg_profile_views = agg_profile_views[agg_profile_views.filter_type.notnull()]

## change column names
agg_profile_views = agg_profile_views.rename(columns = {'ds': 'view_dt', 'cnt': 'view_cnt'})

In [35]:
## add info
analysis_df = agg_profile_views.join(dim_users_clean.set_index('user_id'), on = 'user_id', how = 'left').rename(columns = {'created_at_dt': 'createdAt_Viewer', 'gender': 'gender_Viewer'})
analysis_df = analysis_df.join(dim_users_clean.set_index('user_id'), on = 'viewed_user_id', how = 'left').rename(columns = {'created_at_dt': 'createdAt_Viewed', 'gender': 'gender_Viewed'})
analysis_df = analysis_df.join(dim_bad_actors.set_index('user_id'), on = 'user_id', how = 'left').rename(columns = {'IfBadActor': 'IfBadActor_Viewer'})
analysis_df = analysis_df.join(dim_bad_actors.set_index('user_id'), on = 'viewed_user_id', how = 'left').rename(columns = {'IfBadActor': 'IfBadActor_Viewed'})
analysis_df = analysis_df[['view_dt', 'user_id', 'gender_Viewer', 'IfBadActor_Viewer',
                           'viewed_user_id', 'createdAt_Viewed', 'gender_Viewed', 'IfBadActor_Viewed',
                           'filter_type', 'view_cnt']]

## add tag
analysis_df['view_dt'] = pd.to_datetime(analysis_df['view_dt'])
analysis_df['createdAt_Viewed'] = pd.to_datetime(analysis_df['createdAt_Viewed'])
analysis_df['NewUserViewed_1d'] = analysis_df.apply(lambda x: 1 if x.view_dt == x.createdAt_Viewed else 0, axis = 1)
analysis_df['NewUserViewed_7d'] = analysis_df.apply(lambda x: 1 if x.view_dt < (x.createdAt_Viewed + timedelta(days = 7)) else 0, axis = 1)
analysis_df['IsSameGenderView'] = analysis_df.apply(lambda x: 1 if x.gender_Viewed == x.gender_Viewer else 0, axis = 1)

In [36]:
## output for Tableau use
analysis_df

,view_dt,user_id,gender_Viewer,IfBadActor_Viewer,viewed_user_id,createdAt_Viewed,gender_Viewed,IfBadActor_Viewed,filter_type,view_cnt,NewUserViewed_1d,NewUserViewed_7d,IsSameGenderView
0,2024-08-01,NK+eRjveqeD4O3/+8uQz3uSkRGT55WUUvz4U6HrLVXc=,male,NaN,qZ8H2PQN0HNhsRAAzecOGlIiK7pwyGydyl0CExMBmkc=,2024-08-01,female,NaN,3some,1,1,1,0
1,2024-08-01,J+HgExZqzhwpVHe9M48ABfLJvJ1Z+fdndzM4G5GgG+0=,male,NaN,hl1QlsjB3sIcj6LhoyE5/K13PUhhBLy/LL3OcEpfNKs=,2024-08-01,female,1.0,3some,1,1,1,0
2,2024-08-01,L8wRrEFyK9yMOsqGgGKujbvLgCXdejc5Hu7m+d+wQfo=,male,NaN,GVjxDjZ0z4MyDo/LstQrP+6jp1ti9wBBbdV6/lONkzA=,2024-08-01,female,NaN,3some,1,1,1,0
3,2024-08-01,+oVW3eF18A5ze8m0RKox+e9/TgOptbkdekH3oYEdYgY=,male,NaN,vifKXDPIKswothkT95vBANBajk/Qfj1NqpJs1INGMvQ=,2024-08-01,female,1.0,3some,1,1,1,0
4,2024-08-01,sGI0Ghb0HiIHxSjzsBIAEA6hT/+hTWPbjPMoBhGZ60w=,male,NaN,YXph4dFTUMIhOpySxhm7fxSGSvEbNd44iBFoeocn7pA=,2024-08-01,female,NaN,3some,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1760272,2024-08-31,bHst3J8ylxBc5eKKwAAK0Ifqd/qRTk6ojQ0KdpH+XY8=,male,NaN,YlSxF7FspahZdvx425/wWqWp7HbJ7e6Tv/8LhrhbpTI=,2024-08-01,female,NaN,los_angeles_ca,1,0,0,0
1760273,2024-08-31,Fi3vXveLYh3ki1pfmDEyVARumFfbU87yEMVNIELbOaU=,male,NaN,YlSxF7FspahZdvx425/wWqWp7HbJ7e6Tv/8LhrhbpTI=,2024-08-01,female,NaN,los_angeles_ca,1,0,0,0
1760274,2024-08-31,/CoisblSIgA9vhzx4H4232mBr0hRXnral8iogbiBoqA=,male,NaN,YlSxF7FspahZdvx425/wWqWp7HbJ7e6Tv/8LhrhbpTI=,2024-08-01,female,NaN,los_angeles_ca,1,0,0,0
1760275,2024-08-31,s8ub70VsINbPclbMPL4M1bt5I2bPDa1eNcE+xyv++vg=,male,NaN,YlSxF7FspahZdvx425/wWqWp7HbJ7e6Tv/8LhrhbpTI=,2024-08-01,female,NaN,los_angeles_ca,2,0,0,0


In [37]:
# analysis_df.to_csv('/Users/jimchu/Desktop/DOWN_App/analysis_final.csv', index = False)